In [1]:
import os, sys
import mne, mne_bids
from mne_bids import get_entity_vals, BIDSPath, read_raw_bids
import numpy as np
from scipy.signal import hilbert

basepath = os.path.join(os.getcwd(), "..")
sys.path.append(basepath)
from mne_hfo import LineLengthDetector, RMSDetector, HilbertDetector

In [2]:
%%capture
# this may change depending on where you store the data
root = "C:/Users/patri/Dropbox/fedele_hfo_data"
subjects = get_entity_vals(root, 'subject')
sessions = get_entity_vals(root, 'session')
subjectID = subjects[0]
sessionID = sessions[0]
bids_path = BIDSPath(subject=subjectID, session=sessionID,
                     datatype='ieeg', 
                     suffix='ieeg',
                     extension='.vhdr', root=root)

# get first matching dataset
fpath = bids_path.match()[0]
# load dataset into mne Raw object
extra_params = dict(preload=True)
raw = read_raw_bids(fpath, extra_params)

In [3]:
kwargs = {
    'filter_band': (80, 250), # (l_freq, h_freq)
    'threshold': 3, # Number of st. deviations
    'hfo_name': "ripple"
}
hil = HilbertDetector(**kwargs)

In [4]:
X, y = hil._check_input_raw(raw, None)
print(f"X: {X},\n shape: {X.shape}")

X: [[-6.75505188e-05 -9.27954041e-05 -6.59746887e-05 ... -1.27444067e-04
  -1.25370618e-04 -1.39761438e-04]
 [-8.03429504e-05 -1.10477649e-04 -8.44675232e-05 ... -3.01147400e-05
  -2.04171677e-05 -3.02035980e-05]
 [-1.87599304e-05 -2.46044922e-05 -5.38502731e-06 ... -6.40745056e-05
  -5.67224609e-05 -7.33600525e-05]
 ...
 [-6.74585999e-05 -9.32334412e-05 -6.64845764e-05 ... -9.00949020e-06
  -5.06111259e-06 -2.17972443e-05]
 [-6.01765869e-05 -8.02682678e-05 -5.23955139e-05 ... -5.98103088e-05
  -5.15814087e-05 -6.72028809e-05]
 [-6.70477722e-05 -9.02499023e-05 -6.21961426e-05 ... -8.21194824e-05
  -7.59071899e-05 -9.22437683e-05]],
 shape: (50, 600000)


In [5]:
sfreq = raw.info["sfreq"]

In [8]:
n_windows = 1
hil.win_size = X.shape[1]
hfo_event_arr = np.empty((hil.n_chs, hil.n_bands))
hfo_event_arr.shape

(50, 300)

In [10]:
hil.freq_cutoffs = np.arange(hil.filter_band[0], hil.filter_band[1])
hil.freq_span = (hil.filter_band[1] - hil.filter_band[0]) - 1

In [11]:
sig = X[0, :]

In [12]:
freq_cutoffs = hil.freq_cutoffs
freq_span = hil.freq_span

In [35]:
hfx_bands = []
for ind in range(freq_span):
    l_freq = freq_cutoffs[ind]
    h_freq = freq_cutoffs[ind+1]
    
    signal = mne.filter.filter_data(sig, sfreq=sfreq,
                              l_freq=l_freq, h_freq=h_freq,
                              method='iir', verbose=False)
    
    signal = (signal-np.mean(signal)) / np.std(signal)
    hfx = np.abs(hilbert(signal))
    hfx_bands.append(hfx)
print(f"bands: {hfx_bands}, \nshape: {len(hfx_bands)}, {len(hfx_bands[0])}")

bands: [array([1.48310583, 1.91640966, 2.02588358, ..., 1.08932967, 1.22120833,
       1.18565988]), array([0.15445468, 0.22972896, 0.26311024, ..., 0.05200426, 0.05280661,
       0.07468275]), array([0.21639823, 0.15376693, 0.25903985, ..., 1.08608709, 0.92474367,
       0.58997437]), array([1.49074401, 1.73937202, 1.70538086, ..., 1.79872094, 1.80824621,
       1.52015342]), array([0.4472682 , 0.45371877, 0.39615282, ..., 0.81805257, 0.76789427,
       0.58218658]), array([0.7362222 , 0.74891337, 0.65522896, ..., 1.33885834, 1.25789268,
       0.95483115]), array([0.66342009, 0.59160731, 0.46789161, ..., 1.58521832, 1.4386814 ,
       1.03074351]), array([0.41788852, 0.68104863, 0.81096411, ..., 0.24161546, 0.12410113,
       0.11672427]), array([0.42990954, 0.42537616, 0.36223461, ..., 0.84457448, 0.78594917,
       0.5852854 ]), array([0.38929185, 0.57040434, 0.64341216, ..., 0.12763224, 0.16567487,
       0.21112344]), array([0.25241958, 0.37341727, 0.42291602, ..., 0.08063892, 0.

In [16]:
signal_win_stat = np.empty(freq_span)

In [17]:
signal_win_stat[0] = hfx_bands

ValueError: setting an array element with a sequence.

In [20]:
rms = RMSDetector(**kwargs)

In [24]:
win_start = 0
win_stop = rms.win_size
Xrms, y = rms._check_input_raw(raw, None)
n_windows = rms._compute_n_wins(rms.win_size, rms.step_size, rms.n_times)
n_windows

23997

In [29]:
sig2 = Xrms[0, :]
sig2 = sig2[int(win_start): int(win_stop)]

In [30]:
win_size = rms.win_size
aux = np.power(sig2, 2)
print(aux.shape)
window = np.ones(win_size) / float(win_size)
print(window.shape)
r = np.sqrt(np.convolve(aux, window, 'same'))
print(f"rms: {r}, shape: {r.shape}")

(100,)
(100,)
rms: [4.10541623e-05 4.11580162e-05 4.14298250e-05 4.16053252e-05
 4.19617797e-05 4.23355952e-05 4.26153857e-05 4.28556552e-05
 4.30091964e-05 4.32557932e-05 4.35481755e-05 4.37431140e-05
 4.40775621e-05 4.44169493e-05 4.46350252e-05 4.47760797e-05
 4.48188143e-05 4.49853611e-05 4.51071748e-05 4.51555122e-05
 4.52947920e-05 4.54155933e-05 4.55029261e-05 4.55335408e-05
 4.55337972e-05 4.55783374e-05 4.56041973e-05 4.56360865e-05
 4.56978970e-05 4.57362273e-05 4.57784927e-05 4.57831836e-05
 4.57838938e-05 4.57970278e-05 4.58017550e-05 4.58324762e-05
 4.59075956e-05 4.59406099e-05 4.59635353e-05 4.59642334e-05
 4.59669335e-05 4.60224968e-05 4.60458631e-05 4.61309225e-05
 4.62453370e-05 4.63201965e-05 4.63833879e-05 4.63968578e-05
 4.64472824e-05 4.65685440e-05 4.66381711e-05 4.61463788e-05
 4.52037433e-05 4.47197027e-05 4.41895644e-05 4.33384219e-05
 4.26717857e-05 4.17636802e-05 4.07411957e-05 3.98023613e-05
 3.89371687e-05 3.83444441e-05 3.76334580e-05 3.66824472e-05
 3.59

In [33]:
sig_win_stat = np.empty((n_windows, win_size))
sig_win_stat[0, :] = r

In [34]:
sig_win_stat

array([[4.10541623e-05, 4.11580162e-05, 4.14298250e-05, ...,
        2.33622718e-05, 2.28119609e-05, 2.23437810e-05],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       ...,
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00]])